In [5]:
import numpy as np
import pandas as pd
import os
from libs.utils import *
from libs.tf_idf import *

In [6]:
DATASET_PATH = r"E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\nremove_stopwords.xlsx"

data : pd.DataFrame = read_dataset(DATASET_PATH, "xlsx", header=0)

data.head()

20:49:06 root INFO: Đang đọc dữ liệu từ file E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\nremove_stopwords.xlsx ...
		at Line 330 [read_dataset() in utils.py, utils]
20:49:09 root INFO: Đọc thành công! 
		at Line 338 [read_dataset() in utils.py, utils]


,repair_words,labels
0,"['tiếng', 'anh', 'thi', 'đầu_vào', 'là', 'khôn...",Neutral
1,"['đã', 'từng', 'là', 'niềm', 'mơ_ước', 'khi', ...",Negative
2,"['học_phí', 'hơi', 'cao']",Negative
3,"['̀', '̀', '̣', '̀', '̀', '̉', '̣', '̂', 'đ', ...",Neutral
4,"['thiết_nghĩ', 'đề', 'thi', 'utc', 'nên', 'đổi...",Positive


In [7]:
labels : list[str] = data["labels"].unique().tolist()
labels.sort(reverse=True)
labels

['Positive', 'Neutral', 'Negative']

In [8]:
import string
import re

string_pattele = string.punctuation + '\n'
string_pattele


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_punctuation(text, except_char: str = None):
    try:
        result_text = "".join(
            char for char in text if char not in string_pattele or (except_char and char == except_char))
        result_text = remove_emoji(result_text)
    except TypeError:
        result_text = None
    return result_text

def filter_none_text_id(documents : pd.Series | list[str]) -> list[int]:
    need_filter : list[str] = [text for text in documents]
    none_text_index : list[int] = []
    for i, text in enumerate(need_filter):
        if (text is None):
            print(str(i) + ": " + str(text))
            none_text_index.append(i)
    
    return none_text_index

In [9]:
data["Tokens"] = data["repair_words"].apply(lambda text: [remove_punctuation(x, '_').split()[0] for x in text.split(',')])

In [10]:
corpus: list[list[str]] = [doc for doc in data["Tokens"]]
labels_corpus: list[str] = data["labels"].to_list()

In [11]:
vocabulary = features_extraction(corpus)
vocabulary = list(vocabulary)
vocabulary.sort()

# 0. Nguyên lý của phương pháp sử dụng phân phối tần suất và độ tạp chất để đánh giá từ đặc trưng cho lớp:
## 0.1. Điều kiện:
Khi một từ mang đặc trưng của một lớp (Tức là nhìn vào từ đó có thể đánh giá được văn bản này mang sắc thái lớp nào), từ này sẽ thỏa mãn các điều kiện sau:

* Từ này phải xuất hiện đủ nhiều và phủ rộng trên các văn bản mà thuộc lớp đang xét.

* Từ này chỉ xuất hiện nhiều đối với lớp này mà không xuất hiện nhiều đối với lớp nọ. Hay nói cách khác, từ này mang ý nghĩa phân biệt giữa lớp này với lớp kia.

## 0.2. Phương pháp:

* Đầu tiên, lập các bảng phân phối tần suất của toàn bộ từ điển trên tập dữ liệu ứng với các lớp có trong tập dữ liệu.

* Tính toán độ tạp chất của toàn bộ tập từ điển ứng với các tần suất trích ra từ các bảng.

* Nếu một từ thỏa mãn có tần suất đủ cao tại một lớp và có độ tạp chất đủ nhỏ, từ này sẽ mang đặc trưng cho lớp đó. Ngược lại từ này không mang đặc trưng của lớp nào.


# 1. Xét n-gram = 1:

Tạo phân phối tần suất:

In [12]:
positive_frequency : dict[str, int] = {}
for token in vocabulary:
    positive_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Positive"].values:
    for token in sen:
        positive_frequency[token] += 1

negative_frequency : dict[str, int] = {}
for token in vocabulary:
    negative_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Negative"].values:
    for token in sen:
        negative_frequency[token] += 1

neutral_frequency : dict[str, int] = {}
for token in vocabulary:
    neutral_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Neutral"].values:
    for token in sen:
        neutral_frequency[token] += 1


In [13]:
data[data["labels"] == "Positive"].values[0]


array(["['thiết_nghĩ', 'đề', 'thi', 'utc', 'nên', 'đổi', 'lại', 'tên']",
       'Positive',
       list(['thiết_nghĩ', 'đề', 'thi', 'utc', 'nên', 'đổi', 'lại', 'tên'])],
      dtype=object)

In [14]:
positive_frequency = dict(sorted(positive_frequency.items(), key=lambda x:x[1], reverse=True))
negative_frequency = dict(sorted(negative_frequency.items(), key=lambda x:x[1], reverse=True))
neutral_frequency = dict(sorted(neutral_frequency.items(), key=lambda x:x[1], reverse=True))

Xét độ tạp chất của từng từ trong từ điển:

In [15]:
def gini_score(n_class_samples : list[int]) -> float:
    gini : float = 0
    n_samples : int = 0
    for n in n_class_samples:
        n_samples += n
    for n in n_class_samples:
        gini += (n / n_samples) ** 2
    
    return 1 - gini

In [16]:
gini_score_vocabulary : dict[str, float] = {}
for token in vocabulary:
    gini_score_vocabulary[token] = gini_score([
        positive_frequency[token],
        negative_frequency[token],
        neutral_frequency[token]
    ])

gini_score_vocabulary = dict(sorted(gini_score_vocabulary.items(), key=lambda x:x[1]))


In [17]:
gini_score_vocabulary

{'00': 0.0,
 '01': 0.0,
 '024': 0.0,
 '0396': 0.0,
 '06': 0.0,
 '068': 0.0,
 '06đ': 0.0,
 '07h30': 0.0,
 '0918': 0.0,
 '0936': 0.0,
 '0973': 0.0,
 '0979': 0.0,
 '0981': 0.0,
 '0986': 0.0,
 '1000': 0.0,
 '100m': 0.0,
 '100điểm': 0.0,
 '101': 0.0,
 '102a2': 0.0,
 '1045': 0.0,
 '10700': 0.0,
 '108': 0.0,
 '1096': 0.0,
 '10d': 0.0,
 '10ml': 0.0,
 '10tp': 0.0,
 '10tr': 0.0,
 '110': 0.0,
 '1120': 0.0,
 '1134': 0.0,
 '11r': 0.0,
 '120': 0.0,
 '1256': 0.0,
 '127': 0.0,
 '129m2': 0.0,
 '12h': 0.0,
 '12k': 0.0,
 '139': 0.0,
 '13m2': 0.0,
 '13ml': 0.0,
 '14': 0.0,
 '144hz': 0.0,
 '1470': 0.0,
 '14h30': 0.0,
 '15000': 0.0,
 '15h': 0.0,
 '165hz': 0.0,
 '16h': 0.0,
 '17h30': 0.0,
 '17h45': 0.0,
 '17đ': 0.0,
 '18h': 0.0,
 '18h30': 0.0,
 '18h30h': 0.0,
 '18đ': 0.0,
 '190': 0.0,
 '1900': 0.0,
 '1918': 0.0,
 '192': 0.0,
 '1945': 0.0,
 '1946': 0.0,
 '1949': 0.0,
 '1952': 0.0,
 '1956': 0.0,
 '1962': 0.0,
 '1966': 0.0,
 '1968': 0.0,
 '1972': 0.0,
 '1973': 0.0,
 '1977': 0.0,
 '197m2': 0.0,
 '1982': 0.0,
 '1

In [18]:
THRESHOLD = 1
MIN_SAMPLES = 3

features_vocabulary : dict[str, str] = {} 

for key in gini_score_vocabulary:
    total_samples = positive_frequency[key] + negative_frequency[key] + neutral_frequency[key]
    if gini_score_vocabulary.get(key) <= THRESHOLD and total_samples >= MIN_SAMPLES:
        feq_max = max([
            positive_frequency[key],
            negative_frequency[key],
            neutral_frequency[key],
        ])
        if feq_max == positive_frequency[key]:
            features_vocabulary[key] = "Positive"
        elif feq_max == negative_frequency[key]:
            features_vocabulary[key] = "Negative"
        else:
            features_vocabulary[key] = "Neutral"


In [19]:
features_vocabulary

{'00': 'Neutral',
 '12h': 'Neutral',
 '14': 'Neutral',
 '14h30': 'Neutral',
 '17h30': 'Neutral',
 '18h': 'Neutral',
 '18h30': 'Neutral',
 '1945': 'Neutral',
 '1962': 'Neutral',
 '1h': 'Neutral',
 '2020': 'Neutral',
 '21h30': 'Neutral',
 '24': 'Neutral',
 '248': 'Neutral',
 '25k': 'Neutral',
 '28': 'Neutral',
 '2k4': 'Neutral',
 '2k5': 'Neutral',
 '30k': 'Neutral',
 '360': 'Neutral',
 '42': 'Neutral',
 '4k': 'Positive',
 '4tr': 'Neutral',
 '63': 'Positive',
 '8h30': 'Neutral',
 'acc': 'Neutral',
 'ace': 'Neutral',
 'anh_chị_em': 'Neutral',
 'bch': 'Neutral',
 'bcs': 'Negative',
 'bcvt': 'Positive',
 'biến': 'Neutral',
 'bài_bản': 'Positive',
 'bài_tập_lớn': 'Neutral',
 'bày_tỏ': 'Neutral',
 'bán_nước': 'Negative',
 'báo_cáo': 'Positive',
 'bảo_hiểm_y_tế': 'Neutral',
 'bẩn': 'Negative',
 'bận': 'Neutral',
 'bốt': 'Negative',
 'bổ_ích': 'Positive',
 'bớt': 'Negative',
 'bỡ_ngỡ': 'Neutral',
 'bức_xúc': 'Negative',
 'cao_cấp': 'Neutral',
 'cb': 'Neutral',
 'center': 'Neutral',
 'cetank': 'N

In [117]:
for key in features_vocabulary:
    if features_vocabulary.get(key) == "Negative":
        print(key, end=", ")

bcs, bán_nước, bẩn, bốt, bớt, bức_xúc, chiếm, cmn, cư_xử, cất, cắt, cốp, deadline, dong, dí, giang_hồ, giáo_vụ, grab, gà, gậy, hình_như, khâu, lão, lùa, lũ, may_ra, ngoại_hình, ngón, nhan_sắc, nón, nọ, phản_ánh, quát, soát, sập, thiếu_thốn, trườg, tệ, tổn_thương, web, xước, áo_mưa, ó, đơn_phương, đực, ốm, 1k, dắt, chửi, chặn, im, kêu, tắc, đếm, 2k, già, gọn, kẹt, phi, phụ, yếu, đổ, nhà_xe, 100k, 60, 9tr, chật, chốn, gắt, nhân_phẩm, thừa, vô_duyên, đắt, bà, vụ, 20k, 40, bách_khoa, chương, chịu_đựng, chở, cấm, dừng, hậu, học_kỳ, hổ, khó_chịu, làm_ăn, phóng, sửa, thi_công, tụi, xe_đạp, giải_quyết, leo, kém, cháu, a7, chú, mệt, xe, 54, 70, 90, a6, bố_mẹ, bộ_phận, canh, cs1, cuốn, cán_sự, cổ, cửa, dương, fes, gom, huỷ, hài_lòng, hành_chính, khổ, kéo, liên_thông, lì_xì, lườm, lịch_sự, lỗi, mơ_ước, mạnh_dạn, nuốt, nếu_như, oi, rbc, rác, rưỡi, suy, sáng_rực, sóng, sư_phạm, tay_lái, thậm_chí, thế_thì, trách, trốn, trộm, tát, tường, tỏ, từ_bỏ, tự_dưng, u, vch, vàng, vắng, vỏ, vỡ, vững, ám_ảnh, đ

# 2. Xét n-gram = 2:

In [119]:
vocabulary_2_gram = [(vocabulary[i], vocabulary[i+1]) for i in range(len(vocabulary) - 2)]

In [128]:
positive_frequency_2_gram : dict[tuple[str, str], int] = {}
for token in vocabulary_2_gram:
    positive_frequency_2_gram[token] = 0

for _, label, sen in data[data["labels"] == "Positive"].values:
    for token in vocabulary_2_gram:
        positive_frequency_2_gram[token] += " ".join(sen).count(" ".join(token))

negative_frequency_2_gram : dict[tuple[str, str], int] = {}
for token in vocabulary_2_gram:
    negative_frequency_2_gram[token] = 0

for _, label, sen in data[data["labels"] == "Negative"].values:
    for token in vocabulary_2_gram:
        negative_frequency_2_gram[token] += " ".join(sen).count(" ".join(token))

neutral_frequency_2_gram : dict[tuple[str, str], int] = {}
for token in vocabulary_2_gram:
    neutral_frequency_2_gram[token] = 0

for _, label, sen in data[data["labels"] == "Neutral"].values:
    for token in vocabulary_2_gram:
        neutral_frequency_2_gram[token] += " ".join(sen).count(" ".join(token))


In [129]:
positive_frequency_2_gram = dict(sorted(positive_frequency_2_gram.items(), key=lambda x:x[1], reverse=True))
negative_frequency_2_gram = dict(sorted(negative_frequency_2_gram.items(), key=lambda x:x[1], reverse=True))
neutral_frequency_2_gram = dict(sorted(neutral_frequency_2_gram.items(), key=lambda x:x[1], reverse=True))

In [132]:
neutral_frequency_2_gram

{('0', '00'): 7,
 ('còn', 'có'): 6,
 ('00', '000'): 5,
 ('2k3', '2k4'): 3,
 ('là', 'làm'): 3,
 ('nh', 'nha'): 2,
 ('1', '10'): 1,
 ('26', '27'): 1,
 ('296', '298'): 1,
 ('2k2', '2k3'): 1,
 ('450', '451'): 1,
 ('88', '880'): 1,
 ('a00', 'a01'): 1,
 ('a1', 'a2'): 1,
 ('a7', 'a8'): 1,
 ('cảm_thấy', 'cảm_xúc'): 1,
 ('mái', 'mát'): 1,
 ('nên', 'nó'): 1,
 ('uống', 'v'): 1,
 ('viên', 'viên_chức'): 1,
 ('̂', '̃'): 1,
 ('̛', '̣'): 1,
 ('͎', 'ạ'): 1,
 ('000', '000đ'): 0,
 ('000đ', '01'): 0,
 ('01', '02'): 0,
 ('02', '024'): 0,
 ('024', '03'): 0,
 ('03', '0396'): 0,
 ('0396', '04'): 0,
 ('04', '05'): 0,
 ('05', '06'): 0,
 ('06', '068'): 0,
 ('068', '06đ'): 0,
 ('06đ', '07h30'): 0,
 ('07h30', '08'): 0,
 ('08', '0918'): 0,
 ('0918', '0936'): 0,
 ('0936', '0973'): 0,
 ('0973', '0979'): 0,
 ('0979', '0981'): 0,
 ('0981', '0986'): 0,
 ('0986', '0đ'): 0,
 ('0đ', '1'): 0,
 ('10', '100'): 0,
 ('100', '1000'): 0,
 ('1000', '100k'): 0,
 ('100k', '100m'): 0,
 ('100m', '100điểm'): 0,
 ('100điểm', '101'): 0,


N-gram bằng 2 không có giá trị tìm độ đặc trưng do đó ngừng xét.

# 3. Thử nghiệm với lớp được dùng:

In [11]:
from libs.feature_vocabulary import feature_vocabulary_transfer

feature_vocabulary = feature_vocabulary_transfer(0,0)

feature_vocabulary.fit(corpus, labels_corpus)

TypeError: iteration over a 0-d array

In [4]:
d = {1 : 2, 3 : 4, 5 : 6}

for x in d:
    print(x, d[x])

1 2
3 4
5 6
